# Training Model

This is the **second** notebook of the series.

> Make sure you have executed the previous one [2_Prep.ipynb](2_Prep.ipynb), otherwise this notebook will fail.
---

# Objective

After exploring the data and preparing it, now is the time to train our selected model. The following cells will load the necessary packages and perform the training of our model considering the defined _training_, _test_ and _validation_ sets.

At the end, a trained model will be pickled, so that we could finally use it to Forecast in the final Notebook.